# Microestructura y Sistemas de Trading
## Proyecto Final

Presenta :

- José Armando Melchor Soto


---

### Librerías 

In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import warnings
import datetime as dt
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

---

In [101]:
tickers = yf.download("GFINBURO.MX", start='2019-01-01', end= '2025-01-01', progress=None)['Close']

In [102]:
def get_historical_price(ticker: str, start: str):
    start_dt = pd.to_datetime(start)
    end_dt = start_dt + dt.timedelta(days=4)  # ventana por si no cotiza ese día
    price = pd.DataFrame(yf.Ticker(ticker).history(start=start_dt, end=end_dt)["Close"])
    return float(price.iloc[0, 0])


In [103]:
info = yf.Ticker("GFINBURO.MX")
balance_g = info.balance_sheet.iloc[:,0:-1]
estados_r = info.financials.iloc[:,0:-1]

In [104]:
def get_financial(ticker: str):
    info = yf.Ticker(ticker)
    balance_g = info.quarterly_balance_sheet   # 👈 trimestral
    estados_r = info.quarterly_financials      # 👈 trimestral
    return balance_g, estados_r

balance_g, estados_r = get_financial("GFINBURO.MX")

In [105]:
net_income = estados_r.loc['Net Income Common Stockholders'] 
Shares = balance_g.loc['Ordinary Shares Number']

In [116]:
net_income

2024-12-31    8.843010e+09
2024-09-30    1.002247e+10
2024-06-30    9.305170e+09
2024-03-31             NaN
Freq: -1QE-DEC, Name: Net Income, dtype: float64

In [117]:
Shares

2024-12-31    6.102414e+09
2024-09-30    6.123390e+09
2024-06-30    6.136799e+09
2024-03-31    6.125566e+09
Freq: -1QE-DEC, Name: Shares, dtype: float64

In [106]:
EPS = net_income / Shares
EPS

2023-12-31         NaN
2024-03-31         NaN
2024-06-30    1.524908
2024-09-30    1.642457
2024-12-31    1.453944
2025-03-31    1.290001
2025-06-30    1.373073
Freq: QE-DEC, dtype: float64

In [107]:
mean = EPS.mean()
print(mean.round(4), '%')

1.4569 %


In [108]:
returns = tickers.pct_change().dropna()
trimestre = returns.resample("Q").sum()
trimestre.head()


Ticker,GFINBURO.MX
Date,
2019-03-31,-0.067559
2019-06-30,0.054529
2019-09-30,-0.087377
2019-12-31,-0.074743
2020-03-31,-0.239620


In [119]:
# Align all columns to EPS index
df2 = pd.DataFrame({
    "Quarter": EPS.index,
    "Total Return": trimestre.reindex(EPS.index)["GFINBURO.MX"],
    "Shares Outstanding": Shares.reindex(EPS.index),
    "EPS": EPS
})

df2 

,Quarter,Total Return,Shares Outstanding,EPS
2024-12-31,2024-12-31,-0.021985,6.102414e+09,1.449100
2024-09-30,2024-09-30,0.049223,6.123390e+09,1.636751
2024-06-30,2024-06-30,-0.146467,6.136799e+09,1.516291
2024-03-31,2024-03-31,0.105800,6.125566e+09,NaN
